Quel est le film le plus long ?


Quels sont les 5 films les mieux notés ?

Dans combien de films a joué Morgan Freeman ? Tom Cruise ?

Quels sont les 3 meilleurs films d’horreur ? Dramatique ? Comique ?

Parmi les 100 films les mieux notés, quel pourcentage sont américains ? Français ?

Quel est la durée moyenne d’un film en fonction du genre ?